In [30]:
import os
import keras
import cv2
import pandas as pd
import numpy as np
from skimage.transform import resize
from pathlib import Path
from keras_preprocessing.image import ImageDataGenerator
from  keras.applications import resnet_v2
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from sklearn.model_selection import train_test_split

In [5]:
os.chdir("../../../")
os.getcwd()

'C:\\Users\\tsbau\\git\\idp-radio-1'

'pwd' is not recognized as an internal or external command,
operable program or batch file.


In [23]:
dataset_folder = "data/dev_dataset/"
chexpert_folder = dataset_folder + "CheXpert-v1.0-small/"
DATASET_FOLDER = dataset_folder
SEED = 17

In [24]:
data = pd.read_csv(os.path.join(DATASET_FOLDER + 'train.csv'))

# preprocess
data = data.fillna(0)

# drop lateral images
data = data[~data['Frontal/Lateral'].str.contains("Lateral")]

# drop unrelevant columns
data = data.drop(["Sex", "Age", "Frontal/Lateral", "AP/PA"], axis=1)

# deal with uncertanty (-1) values
data = data.replace(-1,1)

np.random.seed(SEED)
data_train, data_test = train_test_split(data, test_size=0.2)
data_train, data_val = train_test_split(data_train, test_size=0.2)

In [25]:
data_test

,Unnamed: 0,Path,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,patient_id
6392,90680,CheXpert-v1.0-small/train/patient21779/study1/...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21779
13888,199983,CheXpert-v1.0-small/train/patient49541/study4/...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,49541
66,1180,CheXpert-v1.0-small/train/patient00285/study1/...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,285
14041,202038,CheXpert-v1.0-small/train/patient50488/study4/...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,50488
8455,120091,CheXpert-v1.0-small/train/patient28748/study5/...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28748
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8468,120347,CheXpert-v1.0-small/train/patient28801/study1/...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,28801
1655,24190,CheXpert-v1.0-small/train/patient05926/study2/...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,5926
11812,170422,CheXpert-v1.0-small/train/patient39742/study2/...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39742
10214,148159,CheXpert-v1.0-small/train/patient35234/study6/...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,35234


In [26]:
train_datagen=ImageDataGenerator(rescale=1./255)
valid_datagen=ImageDataGenerator(rescale=1./255.)
test_datagen=ImageDataGenerator(rescale=1./255.)

In [27]:
target_size = (224,224)
train_generator=train_datagen.flow_from_dataframe(
    dataframe=data_train, directory=DATASET_FOLDER , x_col='Path', y_col=list(data_train.columns[1:15]), class_mode='other', target_size=target_size, batch_size=32
)
valid_generator = valid_datagen.flow_from_dataframe(
    dataframe=data_val, directory=DATASET_FOLDER , x_col='Path', y_col=list(data_val.columns[1:15]), class_mode='other', target_size=target_size, batch_size=32
)
test_generator = test_datagen.flow_from_dataframe(
    dataframe=data_test, directory=DATASET_FOLDER , x_col="Path", y_col=list(data_test.columns[1:15]), class_mode="other", target_size=target_size, shuffle=False, batch_size=1
)

Found 8556 validated image filenames.
Found 2140 validated image filenames.
Found 2675 validated image filenames.


In [31]:
base_model = resnet_v2.ResNet50V2(include_top=False, weights='imagenet')

# add global pooling and dense output layer 
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
prediction_layer = Dense(14, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=prediction_layer)

62840832/94668760 [==================>...........] - ETA: 15s

KeyboardInterrupt: 

In [ ]:
# freeze all convolutional layers
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# compile model
adam = keras.optimizers.Adam()
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# fit model 
num_epochs = 3
STEP_SIZE_TRAIN= train_generator.n // train_generator.batch_size
STEP_SIZE_VALID= valid_generator.n // valid_generator.batch_size

result = model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=num_epochs)

# save the model
model.save('../../../models/inception/inception-v3.h5')